<a href="https://colab.research.google.com/github/JLuceroVasquez/challenge-telecom-x-latam/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Desafío TelecomX
La empresa TelecomX enfrenta una alta tasa de cancelaciones y necesita comprender los factores que llevan a la pérdida de clientes.

En este desafío se recopiló, procesó y analizó los datos, utilizando Python y sus principales bibliotecas para extraer información valiosa. A partir del análisis, la empresa podrá realizar modelos predictivos y desarrollar estrategias para reducir la evasión.

In [54]:
#Se importan los requisitos
import requests #Para leer los datos del API.
import json #Para cargar los datos del API como una lista de diccionario.
import pandas as pd #Para cargar los datos del API en una dataframe de Pandas.
import numpy as np #Para configurar el tipo de datos en columnas.
import matplotlib.pyplot as plt #Para graficar los diagramas de dispersión y línea de tendencia.
import seaborn as sns #Para graficar los diagramas de boxplot y pointplot.
import plotly.express as px #Para graficar los diagramas de líneas y barras.

##📌 Extracción

In [55]:
#Se almacena la dirección URL de la API en una variable de alcance global.
url = 'https://github.com/ingridcristh/challenge2-data-science-LATAM/raw/refs/heads/main/TelecomX_Data.json'

#Cargamos la consulta a la API en la variable datos_churn.
datos_churn = requests.get(url)
type(datos_churn)

requests.models.Response

In [56]:
#Convertimos los datos de la consulta en texto con el método text.
#Almacenamos los datos leidos como texto en la variable resultado.
resultado = json.loads(datos_churn.text)
#La API transmite los datos en una lista de diccionario, que hemos cargado en la variable resultado.
type(resultado)

list

In [57]:
#Convertimos los datos de la lista de diccionario en un dataframe con la función pd.DataFrame().
df = pd.DataFrame(resultado)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [58]:
#Usamos el método pd.json_normalize para convertir los datos anidados en columnas.
df_plano = pd.json_normalize(resultado)
df_plano.head()

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


##🔧 Transformación

###Conoce el conjunto de datos
Se encontró que ninguna columna tiene valores nulos. Y se cambió el nombre de las columnas para que coincidan con el diccionario de datos.

In [59]:
#Se consulta la cantidad de registros no nulos y el tipo de datos de cada columna.
df_plano.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

In [60]:
#Se renombran las columnas para estar alineados al diccionario de datos.
dict_columnas = {'customer.gender':'gender', 'customer.SeniorCitizen':'SeniorCitizen'
                 ,'customer.Partner':'Partner', 'customer.Dependents':'Dependents',
                 'customer.tenure':'tenure', 'phone.PhoneService':'PhoneService',
                 'phone.MultipleLines':'MultipleLines', 'internet.InternetService':'InternetService',
                 'internet.OnlineSecurity':'OnlineSecurity', 'internet.OnlineBackup':'OnlineBackup',
                 'internet.DeviceProtection':'DeviceProtection', 'internet.TechSupport':'TechSupport',
                 'internet.StreamingTV':'StreamingTV', 'internet.StreamingMovies':'StreamingMovies',
                 'account.Contract':'Contract', 'account.PaperlessBilling':'PaperlessBilling',
                 'account.PaymentMethod':'PaymentMethod', 'account.Charges.Monthly':'Charges.Monthly',
                 'account.Charges.Total':'Charges.Total'}

df_plano.rename(dict_columnas, axis=1, inplace=True)
df_plano.columns

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly',
       'Charges.Total'],
      dtype='object')

###Comprobación de incoherencia en los datos
En esta etapa se prestó atención a valores nulos, ausentes, duplicados, errores de formato e inconsistencias en las categorías. Aunque no hay valores duplicados y no hay valores nulos, se encontraron las siguientes incoherencias:
- **Columnas**: Los nombres tienen mayusculas, minúsculas y puntos.
- **`Churn`**: Tiene 224 registros con valores ausentes.
- **`Charges.Total`**: Tiene 11 registros con valores ausentes. También tiene error de formato. Debe ser de tipo float.

Además, para facilitar el procesamiento matemático y comprensión de la información, se realizó:
- **Traducción**: A español de las columnas `gender`, `PaymentMethod` y `Contract`.
- **Cambio del tipo de dato**: A bool de las columnas `SeniorCitizen`, `Partner`, `Dependents`, `PhoneService`, `MultipleLines`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`,` StreamingTV`, `StreamingMovies` y `PaperlessBilling`.



####Valores únicos en cada columna

In [61]:
#Se consulta los valores únicos en cada columna, y sin son menores de 10 se muestran.
for columna in df_plano.columns:
  valores = df_plano[columna].unique()
  cantidad = len(valores)
  print(f'La columna "{columna}" tiene {cantidad} valores únicos.')
  if cantidad < 10:
    print(valores)
    print('-'*50)

La columna "customerID" tiene 7267 valores únicos.
La columna "Churn" tiene 3 valores únicos.
['No' 'Yes' '']
--------------------------------------------------
La columna "gender" tiene 2 valores únicos.
['Female' 'Male']
--------------------------------------------------
La columna "SeniorCitizen" tiene 2 valores únicos.
[0 1]
--------------------------------------------------
La columna "Partner" tiene 2 valores únicos.
['Yes' 'No']
--------------------------------------------------
La columna "Dependents" tiene 2 valores únicos.
['Yes' 'No']
--------------------------------------------------
La columna "tenure" tiene 73 valores únicos.
La columna "PhoneService" tiene 2 valores únicos.
['Yes' 'No']
--------------------------------------------------
La columna "MultipleLines" tiene 3 valores únicos.
['No' 'Yes' 'No phone service']
--------------------------------------------------
La columna "InternetService" tiene 3 valores únicos.
['DSL' 'Fiber optic' 'No']
------------------------

####Registros nulos

In [62]:
#Se imprime la cantidad de registros nulos.
print(f'Cantidad de registros nulos:\n{df_plano.isnull().sum()}')

Cantidad de registros nulos:
customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Charges.Monthly     0
Charges.Total       0
dtype: int64


####Registros duplicados

In [63]:
#Se imprime la cantidad de registros duplicados.
print('La cantidad de registros duplicados son:',df_plano.duplicated().sum())

La cantidad de registros duplicados son: 0


####Registros vacíos

In [64]:
#Se calcula la cantidad de registros vacíos por columna.
'''
Se elimina los espacios vacíos al inicio y final de cada valor de la columna.
Previamente se convierten los valores a string para poder aplicar el .str y
consecuentemente el .strip().
Lo anterior devuelve una serie de Pandas con valores booleanos, donde True
coincide con celdas vacías y que al aplicarse el método.sum() se suman como 1.
Como resultado, se obtiene una serie cuyos índices son el nombre de columnas y
los valores son la cantidad de celdas vacías.
'''
df_plano.apply(lambda x: x.astype(str).str.strip()=='').sum()

,0
customerID,0
Churn,224
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0


###Manejo de inconsistencias
- **Columnas**: Todos los nombres están en minúsculas y separados por guiones bajos.
- **`Churn`**: No tiene registros con valores ausentes.
- **`Charges.Total`**: Es de tipo float, teniendo como valor NaN en los registros donde hubieron valores ausentes.

####Nombres de columnas

In [65]:
#Se escriben los nombres de las columnas en minúsculas y con guiones bajos en lugar de espacios.
df_plano.columns = df_plano.columns.str.lower().str.replace('.', '_')
df_plano.columns

Index(['customerid', 'churn', 'gender', 'seniorcitizen', 'partner',
       'dependents', 'tenure', 'phoneservice', 'multiplelines',
       'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection',
       'techsupport', 'streamingtv', 'streamingmovies', 'contract',
       'paperlessbilling', 'paymentmethod', 'charges_monthly',
       'charges_total'],
      dtype='object')

####Columna **`Churn`**

In [66]:
#Se eliminan los registros con valores vacíos en Churn mediante indexación booleana.
df_plano = df_plano[df_plano.churn !='']
#Se consulta la cantidad de registros según el valor en la columna churn.
df_plano.churn.value_counts()

,count
churn,
No,5174
Yes,1869


####Columna **`Charges.Total`**

In [67]:
#En la etapa anterior, se conoció que hay 11 registros vacíos.
#Se eliminan espacios al inicio y final de cada registro.
df_plano.charges_total = df_plano.charges_total.str.strip()

/tmp/ipython-input-67-3590232982.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_plano.charges_total = df_plano.charges_total.str.strip()


In [68]:
#Se corrobora cuantos registros de la columna contienen caracteres especiales o están vacíos.
'''
Se utiliza dos condiciones en una sola expresión regular:

  - [^\d.]
    para identificar caractéres que no sean números o 1 punto.

  - (?=(?:.*\\.){2,})
    para identificar la presencia múltiples puntos.

  - ^$
    para identificar si el registro está vacío.
'''
regex = r'[^\d.]|(?=(?:.*\.){2,})|^$'
df_plano['charges_total'].str.contains(regex, regex=True).sum()

np.int64(11)

In [69]:
#Se encontraron 11 registros con formatos problemáticos, se visualizan para ver si se pueden rescatar.
df_plano.charges_total[df_plano.charges_total.str.contains(regex, regex=True)]

,charges_total
975,
1775,
1955,
2075,
2232,
2308,
2930,
3134,
3203,
4169,


In [70]:
#Se comprueba que los 11 registros están vacíos, y se procede con la conversión.
'''
El parámetro coerce indica que los errores de conversión serán tratados como valore nulos.
'''
df_plano.charges_total = pd.to_numeric(df_plano.charges_total, errors='coerce', downcast='float')
df_plano.charges_total.dtypes

/tmp/ipython-input-70-751594120.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_plano.charges_total = pd.to_numeric(df_plano.charges_total, errors='coerce', downcast='float')


dtype('float32')

In [71]:
df_plano.charges_total[df_plano.charges_total.isna()]

,charges_total
975,NaN
1775,NaN
1955,NaN
2075,NaN
2232,NaN
2308,NaN
2930,NaN
3134,NaN
3203,NaN
4169,NaN


###Estandarización y transformación de datos
- **Se tradujo**: A español de las columnas `gender`, `PaymentMethod` y `Contract`.
- **Se cambio el tipo de dato**: A bool de las columnas `SeniorCitizen`, `Partner`, `Dependents`, `PhoneService`, `MultipleLines`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`,` StreamingTV`, `StreamingMovies` y `PaperlessBilling`.

####Traducción al español de columnas

In [72]:
#Se remplazaron los valores de la columna gender por su traducción en español.
df_plano.gender = df_plano.gender.apply(lambda x: x.replace('Female','Femenino').replace('Male','Masculino'))
df_plano.gender.unique()

/tmp/ipython-input-72-1525351479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_plano.gender = df_plano.gender.apply(lambda x: x.replace('Female','Femenino').replace('Male','Masculino'))


array(['Femenino', 'Masculino'], dtype=object)

In [73]:
#Se remplazaron los valores de la columna paymentmethod por su traducción en español.
df_plano.paymentmethod = df_plano.paymentmethod.apply(lambda x: x.replace('Mailed check','Cheque por correo').
                                                      replace('Electronic check','Cheque electrónico').
                                                      replace('Credit card (automatic)','Tarjeta de crédito (automático)').
                                                      replace('Bank transfer (automatic)','Transferencia bancaria (automático)'))
df_plano.paymentmethod.unique()

array(['Cheque por correo', 'Cheque electrónico',
       'Tarjeta de crédito (automático)',
       'Transferencia bancaria (automático)'], dtype=object)

In [74]:
#Se remplazaron los valores de la columna contract por su traducción en español.
df_plano.contract = df_plano.contract.apply(lambda x: x.replace('One year','Un año').
                                            replace('Month-to-month', 'Mes a mes').
                                            replace('Two year', 'Dos años'))
df_plano.contract.unique()

array(['Un año', 'Mes a mes', 'Dos años'], dtype=object)

####Tipado de columnas booleanas

In [75]:
#Se convierten los valores de la columna "seniorcitizen".
'''
Se empleó el método .astype porque la columna tiene valores numéricos 0 y 1 que
Pandas traduce automáticamente en False y True.
'''
df_plano.seniorcitizen = df_plano.seniorcitizen.astype('bool')
df_plano.seniorcitizen.dtypes

dtype('bool')

In [76]:
#Se convierten los valores de 11 columnas cuyos valores únicos fueron 'yes' y 'no'.
columnas_bool = ['partner', 'dependents', 'phoneservice', 'multiplelines',
                 'onlinesecurity', 'onlinebackup', 'deviceprotection',
                 'techsupport','streamingtv', 'streamingmovies',
                 'paperlessbilling']
df_plano[columnas_bool] = df_plano[columnas_bool].map(lambda x: True if x == 'Yes' else False).astype('bool')
df_plano[columnas_bool].dtypes

,0
partner,bool
dependents,bool
phoneservice,bool
multiplelines,bool
onlinesecurity,bool
onlinebackup,bool
deviceprotection,bool
techsupport,bool
streamingtv,bool
streamingmovies,bool


###Columnas de cuentas diarias y servicios contratados

####Columna nueva: Cuenta diaria

In [77]:
#Se calculan los valores de la columna nueva cuenta_diaria.
df_plano['cuenta_diaria'] = df_plano.charges_monthly / 30
df_plano['cuenta_diaria'].dtypes

dtype('float64')

####Columna nueva: Servicios contratados

In [78]:
#Se calculan los valores de la columna nueva servicios_contratados.
'''
Se suma los valores en las columnas del tipo servicio con los valores tipo bool
de una copia temporal de la columna internet.
'''
columnas_servicios = ['phoneservice', 'multiplelines', 'onlinesecurity',
                      'onlinebackup', 'deviceprotection', 'techsupport',
                      'streamingtv', 'streamingmovies']

df_plano['servicios_contratados'] = df_plano[columnas_servicios].sum(axis=1) + df_plano.internetservice.apply(lambda x: True if x in {'DSL', 'Fiber optic'} else False)
df_plano['servicios_contratados'].dtypes

dtype('int64')

##📊 Carga y análisis

###Análisis descriptivo

###Distribución de evasión con Matplotly

###Recuento de evasión por categoría

####Gráficos de barras con Matplotly

####Gráficos de puntos con Seaborn
Se realizaron gráficos de puntos para las **variables de negocio** `PaymentMethod`, `Contract` y `PaperlessBilling` en combinación con las **variables demográficas** `gender`, `SeniorCitizen`, `Partner` y `Dependents`.

###Conteo de evasión por variable numérica

####Gráficos de cajas y bigotes con Plotly

####Gráficos de líneas con Plotly
Se realizaron gráficos de lineas de la `probabilidad churn` en función de **variables numéricas** como `tenure`, `cuenta_diaria` y `servicios_contratados`.

###Gráficos de dispersión y correlación

####Correlación entre % Churn y Tenure

####Correlación entre % Churn y Cuenta diaria

####Correlación entre % Churn y Cantidad de servicios contratados

##📄Informe final